# Assignment Machine Learning
***Assignment BIT07 / Bioinformatics @ home***
For this assignment, a dataset on breast cancer characteristics, RNA expression and survival was chosen, because of the medical background 

This dataset has a mixture of categorical variables (both ordinal and nominal) and numerical variables. It has the possibility to predict survival in both 

In [ ]:
# Importeren van alle modules, functies en methodes
###################################################
# Data analyse en wetenschappelijke berekeningen
from scipy import stats
import numpy as np
import pandas as pd

# Data visualisatie (1 of beide; seaborn is mooi alternatief)
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import rcParams
rcParams['figure.figsize'] = 20,8.27 #grootte van de figuren in inch

# Machine learning modules
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn import preprocessing
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import confusion_matrix, accuracy_score, roc_curve, auc, det_curve, balanced_accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer


# Eigen functies
################
def listdiff(li1, li2):
    li_dif = [i for i in li1 + li2 if i not in li1 or i not in li2]
    return li_dif


def ohe_and_bind(original_dataframe, features_to_encode):
    if type(features_to_encode) is not list:
        raise Exception("Features to encode needs to be a list")

    for f_encode in features_to_encode:
        dummies = pd.get_dummies(original_dataframe[[f_encode]])
        original_dataframe = original_dataframe.drop(f_encode, axis=1)
        original_dataframe = pd.concat([original_dataframe, dummies], axis=1)

    return(original_dataframe)

## Stap 1: Inlezen van de dataset en vooranalyse
Volgende stappen moeten worden doornomen:
1. Inlezen van de dataset
2. Beschrijving van de dataset
3. Zoeken naar missing values
4. Zoeken naar inconsistente waarden
5. Zoeken naar uitschieters
6. Verwijderen of omvormen van data

Op het einde een beslissing maken of je deze waarden verwijderdt, behoudt of omvormt.

Je kan in 2 richtingen verwijderen: features en observaties. Als de vreemde waarden vooral binnen 1 feature voorvalt, dan is het beter deze kolom te verwijderen. Als de vreemde waarden eerder per observatie/patiënt wordt gezien is het beter om rijen te verwijderen. Bij twijfel eerder rijen verwijderen. Zorg dat er op het einde genoeg rijen overblijven. 

Na alle data gecontroleerd te hebben, moet je features en targets vaststellen (voor zowel regressie als classificiatie).

In [ ]:
# Stap 1.1: Inlezen van de dataset
df = pd.read_csv('file.csv')
df.head()

In [ ]:
# Stap 1.2: beschrijven van de dataset
df.shape #dimensies van de dataset
df.describe() # range van waarden; vul ook aan met info over kolommen van bron

In [ ]:
# Stap 1.3: Zoeken naar missing values
df.isnull().sum().sort_values(ascending=False)

In [ ]:
# Stap 1.4: Zoeken naar inconsistente
df.describe() # range van waarden kloppen met mogelijke werkelijkheid? Te veel nullen als missing values?

In [ ]:
# Stap 1.5: Zoeken naar uitschieters
columns = df.columns()

for column in columns:
    ax1 = sns.boxplot(x=column, data=df) #interessanter voor categorische data
    ax2 = sns.histplot(x=column, data=df) #beter voor numerieke data

In [ ]:
# Stap 1.6: Verwijderen van data
df.drop('column', axis=1, inplace=True) #kolom verwijderen wegens niet contributief of missing values
df.drop(df[(df.column1 == 0) | (df.column2 == 0) | (df.column3 == 0)].index, inplace = True) #Rijen verwijderen met foutieve waarde 0
df = df[(np.abs(stats.zscore(df)) < 5).all(axis=1)] #Verwijderen van uitschieters die meer dan 5x SD verwijderd zijn van gemiddelde
df = df.dropna() #Verwijderen van rijen met missing values

### Conclusie stap 1
Deze dataset bestaat uit volgende kolommen (+info):
- Kolom 1: info
- Kolom 2: info
- ...

Er zijn X observaties en Y kolommen. Hierin zijn er A vreemde waarden/missing values. Deze worden verwijderd/behouden/omvormd. Omvormen (imputing) kan pas na het splitsen van de dataset.

## Stap 2: Analyse en preprocessing
**Stap 2.1: analyse**  
Bij de analytische fase is het de bedoeling om te kijken naar welke features een invloed hebben op het target. Dit kan op volgende manieren:
- Pairplot (moeilijk leesbaar bij veel features --> exporteren en zoomen)
- Heatmap (moeilijk leesbaar bij veel features --> exporteren en zoomen)
- Afzonderlijke scatterplots
- Boxplots van categorieke features
- Gebalanceerdheid van de data

Creativiteit de vrije loop laten gaan!

**Stap 2.2 preprocessing**  
Voor het preprocessen zijn er verschillende stappen te ondergaan:
1. Omvormen van ordinale categorieën (ordinal encoding)
2. Omvormen van categorische waarden (one-hot encoding)
3. Opsplitsen in features en targets
4. Opsplitsen in testset en trainingsset
4. Omvormen van missing values (imputing)
5. Schalen van numerieke features

Bij elke stap laten weten op welke manier en waarom je dit doet.

In [ ]:
# Stap 2.1.1: Pairplot
sns.pairplot(df)

In [ ]:
# Stap 2.1.2 Heatmap
df_corr = df.corr()
f, ax = plt.subplots(figsize=(10, 8))
sns.heatmap(df_corr, 
            mask=np.zeros_like(df_corr, dtype=bool), 
            cmap=sns.diverging_palette(220, 10, as_cmap=True),
            square=True, 
            ax=ax,annot=True
           )

In [ ]:
# Stap 2.1.3: Afzonderlijke scatterplots
sns.scatterplot(data=df, x='feature', y='target')

In [ ]:
# Stap 2.1.4: Boxplots van categorieke waarden
sns.boxplot(x=df["feature"], y=df["target"])

In [ ]:
# Stap 2.1.5: Gebalanceerdheid van de data op basis van target
df['target'].value_counts() #absolute waarden
df['target'].value_counts(normalize=True) #percentages
sns.countplot(x="target", data=df) #histogram

### Conclusie stap 2.1
Formuleer enkele bevindingen van de data-analyse. Kopieer deze ook al naar de eindconclusie.

In [ ]:
# Stap 2.2.1: Omvormen ordinale categorieën: rankings naar nummers
cat1 = ['rank1', 'rank2', 'rank3'] # categoriesleutel op basis van de vooranalyse voor feature 1
cat2 = ['rank1', 'rank2', 'rank3'] # categoriesleutel op basis van de vooranalyse voor feature 2

ord_features = ['feature1', 'feature2'] # lijst met alle colomnamen met ordinale features
rest_features = listdiff(df.columns, ord_features)

ordinal_trf = ColumnTransformer(
    transformers =[
        ('ord', #gewoon een naam
         preprocessing.OrdinalEncoder(categories=[cat1, cat2]), # Belangrijk!: er moeten evenveel categorieën zijn als aantal om te zetten features. Deze moeten in volgorde staan en als twee features zelfde categorieën hebben mag dit herhaald worden
         ord_features), #Lijst met de features om te transformeren
], remainder ='passthrough')

df = pd.DataFrame(ordinal_trf.fit_transform(df), columns=[ord_features, rest_features])

In [ ]:
# Stap 2.2.2: One-hot encoding
ohe_features = [featureA, featureB]
df = ohe_and_bind(df, ohe_features)

In [ ]:
# Stap 2.2.3: Opsplitsen in features en target
y = df['target'].values.astype('int')
X = pd.DataFrame(df.drop('target', axis=1))

In [ ]:
# Stap 2.2.4: Opsplitsen in training en testset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=200, random_state=0)

In [ ]:
# Stap 2.2.5: Imputing (kan wegvallen indien niet nodig)
imp = SimpleImputer(missing_values=np.nan, strategy='mean') # of andere strategie
imp.fit(X_train)

imp.transform(X_train)
X_train = pd.DataFram(X_train, columns=imp.feature_names_in_)
imp.transform(X_test)
X_test = pd.DataFram(X_test, columns=imp.feature_names_in_)

In [ ]:
# Stap 2.2.6: Schaling (moet gebeuren!)
scaler = StandardScaler() # of andere schaler
scaler.fit(X_train)
scaler.transform(X_train)
scaler.transform(X_test)

### Conclusie stap 2.2
Maak een conclusie over alle preprocessing eventueel met vermelding van het aantal features nu. Ook altijd vermelden waarom je voor welke techniek hebt gekozen.

## Stap 3: Regressie
Gebruik lineaire regressie om een conintue variabele target te voorspellen.

Stappenplan:
1. Initieel model zonder tweaking
2. Initieel model beoordelen (MSE, MAE en R²)
3. Model tunen
    1. Nieuwe features creëren
    2. Polynomiale expansie
    3. Regularisatie
    4. Hyperparameter tuning: Grid search/Random search
4. Model selecteren op basis van Grid search
5. Finaal model evalueren

Het is moeilijk om hier voorbeeldcode te geven. Dit moet bijna altijd worden aangepast aan de situatie.

## Stap 4: Classificatie
Gebruik Logistische regressie, SVM om een categorieke variabele target te voorspellen.

Stappenplan (zelfde als bij lineaire regressie, maar meerdere type modellen te testen bij Grid search):
1. Initieel model zonder tweaking
2. Initieel model beoordelen (Precisie, Recall, Accuracy, Confusiematrix, ROC en AUC)
3. Model tunen
    1. Nieuwe features creëren
    2. Polynomiale expansie
    3. Regularisatie
    4. Hyperparameter tuning: Grid search/Random search
4. Model selecteren op basis van Grid search
5. Finaal model evalueren

Het is moeilijk om hier voorbeeldcode te geven. Dit moet bijna altijd worden aangepast aan de situatie. Bij Classificatie is het altijd leuk om de ROC curves te tekenen.

## Stap 5: Clustering
Gebruik een clustering methode om een cluster te voorspellen.

Stappenplan (zelfde als bij lineaire regressie, maar meerdere type modellen te testen bij Grid search):
1. Initieel model zonder tweaking
2. Initieel model beoordelen (nog op te zoeken: evaluatieparameters)
3. Model tunen
    1. Nieuwe features creëren
    2. Polynomiale expansie
    3. Regularisatie
    4. Hyperparameter tuning: Grid search/Random search (? geen idee of dit past)
4. Model selecteren op basis van Grid search
5. Finaal model evalueren

Het is moeilijk om hier voorbeeldcode te geven. Dit moet bijna altijd worden aangepast aan de situatie. We moeten blijkbaar niet alle attributen gebruiken in het model. We mogen ons beperken tot de attributen met een goede performance of een combinatie van verschillende goede attributen. We moeten het wel proberen te baseren op dezelfde attributen als bij classificatie.

In [ ]:
# Stap x.1: Initieel model
model = module.Methode(parameters) # Aan te passen voor het type model je wilt gebruiken
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

## Voorbeelden modellen:
lregmodel = linear_model.LinearRegression()
ridgemodel = linear_model.Ridge(alpha=alpha,tol=0.0001,fit_intercept=True) # L2 regularisatie
lasso = linear_model.Lasso(alpha=alpha,tol=0.0001,fit_intercept=True) # L1 regularisatie
logrmodel = linear_model.LogisticRegression(C=1, solver='liblinear', class_weight='balanced', multi_class='ovr')
svmmodel = svm.SVC(kernel='linear',C=0.01)
# Wordt nog verder aangevuld

In [ ]:
# Stap x.2: Model beoordelen
## Linear, Ridge en Lasso
r2train = lregmodel.score(X_train, y_train)
r2test = lregmodel.score(X_test, y_test)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

## Logistic en SVM
report = classification_report(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)*100
matrix = confusion_matrix(y_test, y_pred)

## Clustering
# Wordt nog aangevuld

In [ ]:
# Stap x.3: Model tunen (werk opnieuw vanaf de basisset)
## x.3.1: Nieuwe features creëren
df.insert(df.columns.size-1, 'new_feature', df.feature1*df.feature2) # kan ook som, min, deling


## x.3.2: Polynomiale expansie
pol = df[['lijst met features met continue variabelen']]
nopol = lego.drop(['lijst met features met continue variabelen'], axis=1)

graad = 4 

poly = PolynomialFeatures(graad)
poly.fit(pol)
polynom = pd.DataFrame(poly.transform(pol), columns=poly.get_feature_names_out(pol.columns))

### Belangrijk! vul aan met One-hot encoding en ordinal encoding indien nodig, daarna opnieuw opsplitsen
ord_features = ['feature1', 'feature2'] # lijst met alle colomnamen met ordinale features
rest_features = listdiff(df.columns, ord_features)
nopol = pd.DataFrame(ordinal_trf.fit_transform(nopol), columns=[ord_features, rest_features])

ohe_features = [featureA, featureB]
nopol = ohe_and_bind(nopol, ohe_features)

df_preprocessed = pd.concat([polynom, nopol], axis=1)

y = df_preprocessed['target'].values.astype('int')
X = pd.DataFrame(df_preprocessed.drop('target', axis=1))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=200, random_state=0)


## X.3.3: Regularisatie (kan ook tijdens gridsearch)
### Linear regression --> gebruik Lasso of Ridge regressie
### Logistic regression --> standaard is L2
lr = linear_model.LogisticRegression(C=1, solver='liblinear', class_weight='balanced', penalty='l1')
### SVM heeft geen regularisatie maar wel kernels
svmmodel = svm.SVC(kernel='poly',C=10000000)


## X.3.4: Hyperparameter tuning met grid search of random search
### linear regression alleen bij ridge/lasso en weinig zin
lrparam = {'alpha' = np.logspace(-5, 4, 10)}

grid_search = GridSearchCV(estimator = model, 
                           param_grid = lrparam,
                           scoring = 'r2', # roc_auc, f1_weighted, f1_macro, recall, ...
                           cv = 15,
                           n_jobs = -1,
                           verbose = 5)

best_accuracy = grid_search.best_score_ 
best_parameters = grid_search.best_params_  
y_pred = grid_search.predict(X_test)
r2train = lregmodel.score(X_train, y_train)
r2test = lregmodel.score(X_test, y_test)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)


### logistic regression grid search
logrparam = {
    'solver' = ['newton-cg', 'lbfgs', 'liblinear'],
    'penalty' = ['none', 'l1', 'l2', 'elasticnet'],
    'C' = np.logspace(-5, 4, 10)
}

grid_search = GridSearchCV(estimator = model, 
                           param_grid = logrparam,
                           scoring = 'accuracy', # roc_auc, f1_weighted, f1_macro, recall, ...
                           cv = 15,
                           n_jobs = -1,
                           verbose = 5)

grid_search = grid_search.fit(X_train, y_train)

best_accuracy = grid_search.best_score_ 
best_parameters = grid_search.best_params_  
y_pred = grid_search.predict(X_test)
report = classification_report(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)*100
matrix = confusion_matrix(y_test, y_pred)

### logistic regression random search
logrparam = {
    'solver' = ['newton-cg', 'lbfgs', 'liblinear'],
    'penalty' = ['none', 'l1', 'l2', 'elasticnet'],
    'C' = loguniform(1e-5, 100)
}

random_search = RandomizedSearchCV(model, 
                                   logrparam, 
                                   n_iter=20, 
                                   scoring='accuracy', 
                                   n_jobs=-1, 
                                   cv=5, 
                                   random_state=1)

random_search = random_search.fit(X_train, y_train)

best_accuracy = random_search.best_score_ 
best_parameters = random_search.best_params_  
y_pred = random_search.predict(X_test)
report = classification_report(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)*100
matrix = confusion_matrix(y_test, y_pred)

### SVM grid search
svmparamaters = [ 
        {'kernel': ['linear'], 'C': np.linspace(0.01,20,10)},
        {'kernel': ['rbf'], 'C': np.linspace(0.01,20,10), 'gamma': [0.0001, 0.001, 0.01, 0.1, 0.2]},
        {'kernel': ['poly'], 'C':np.linspace(0.01,20,10)} ]

grid_search = GridSearchCV(estimator = model, 
                           param_grid = svmparamaters,
                           scoring = 'accuracy', # roc_auc, f1_weighted, f1_macro, recall, ...
                           cv = 15,
                           n_jobs = -1,
                           verbose = 5)

grid_search = grid_search.fit(X_train, y_train)

best_accuracy = grid_search.best_score_ 
best_parameters = grid_search.best_params_  
y_pred = grid_search.predict(X_test)
report = classification_report(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)*100
matrix = confusion_matrix(y_test, y_pred)

### SVM random search
model = SVC(probability=True)
parameters = {'kernel': ['linear','rbf','poly'],
              'C': uniform(0.01, 20), # haal C uit een random uniform distribution
              'gamma': uniform(0.001, 0.2)}


n_iter_search = 20

random_search = RandomizedSearchCV(model, param_distributions=parameters,cv=5,n_iter=n_iter_search,n_jobs = -1,verbose=1)

random_search = random_search.fit(X_train, y_train)

best_accuracy = random_search.best_score_ 
best_parameters = random_search.best_params_  
y_pred = random_search.predict(X_test)
report = classification_report(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)*100
matrix = confusion_matrix(y_test, y_pred)

#### Nog aan te vullen voor clustering

## Conclusion
Conclusie schrijven over:
- Dataset
    - Goed/slecht en waarom?
    - Problemen in de dataset
- Model
    - Beste model voor elke techniek
    - Performance
    - Overfitting/underfitting
    - Kan gebruikt worden in dagdagelijkse praktijk?